In [1]:
# 공동주택가격정보 자료(like apt, yeonlip)
# 공동주택가격정보 자료(like dandok)
# http://www.nsdi.go.kr/lxportal/?menuno=2679
# made by Beans Im

In [ ]:
import pandas as pd
import geopandas as gpd
import glob
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [ ]:
pbar = ProgressBar()
pbar.register()

In [19]:
paths = glob.glob('D:/feature_data/개별주택가격정보/*.csv')
# ['고유번호', '법정동코드', '법정동명', '특수지구분코드', '특수지구분명', '지번', '건축물대장고유번호', '기준연도',
# '기준월', '동코드', '동명', '토지대장면적', '산정대지면적', '건물전체연면적', '건물산정연면적', '주택가격',
# '표준지여부', '데이터기준일자']

"\n['고유번호', '법정동코드', '법정동명', '특수지구분코드', '특수지구분명', '지번', '건축물대장고유번호', '기준연도',\n'기준월', '동코드', '동명', '토지대장면적', '산정대지면적', '건물전체연면적', '건물산정연면적', '주택가격',\n'표준지여부', '데이터기준일자']\n"

In [33]:
dandok = pd.DataFrame()
for path in paths:
    print(path)
    tmp = dd.read_csv(
        path, 
        usecols=['법정동명', '지번', '기준연도', '기준월', '주택가격'],
        dtype={'지번': 'str', '기준연도': 'str', '기준월': 'str'},
        encoding = 'cp949'
    )
    tmp = tmp[(tmp['기준연도'].isin(['2021']))].compute()
    public = pd.concat([dandok, tmp], ignore_index=True)
    
dandok.drop_duplicates(inplace = True)
dandok.columns = ['dong_address', 'bunji', 'registry_pk', 'base_year', 'base_month','gov_price']    
dandok = dandok[dandok['base_month'] == '01'].reset_index(drop=True)
dandok['address_dong'] = dandok['dong_address'] + ' ' + dandok['bunji'] + '번지'
dandok['address_dong'] = dandok['address_dong'].str.strip()
dandok = dandok.drop(['dong_address', 'bunji', 'base_month'], axis=1)
print(dandok.shape)

D:/feature_data/개별주택가격정보\AL_11_D169_20210714.csv
[                                        ] | 0% Completed |  0.1s

C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\dataframe\io\csv.py:653: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  return read_pandas(
C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\dataframe\io\csv.py:653: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  return read_pandas(
C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\core.py:121: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  return func(*(_execute_task(a, cache) for a in args))
C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\core.py:121: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed |  8.3s
D:/feature_data/개별주택가격정보\AL_26_D169_20210714.csv
[                                        ] | 0% Completed |  0.1s

C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\dataframe\io\csv.py:653: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  return read_pandas(
C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\dataframe\io\csv.py:653: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  return read_pandas(
C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\core.py:121: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  return func(*(_execute_task(a, cache) for a in args))
C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\core.py:121: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed |  4.4s
D:/feature_data/개별주택가격정보\AL_27_D169_20210714.csv
[                                        ] | 0% Completed |  0.1s

...

[########################################] | 100% Completed | 11.0s
D:/feature_data/개별주택가격정보\AL_50_D169_20210714.csv
[                                        ] | 0% Completed |  0.1s

C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\dataframe\io\csv.py:653: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  return read_pandas(
C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\dataframe\io\csv.py:653: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  return read_pandas(


[                                        ] | 0% Completed |  0.6s

C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\core.py:121: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  return func(*(_execute_task(a, cache) for a in args))
C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\core.py:121: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed |  2.7s


In [43]:
paths = glob.glob('D:/feature_data/공동주택가격정보/*.csv')
# ['고유번호', '법정동코드', '법정동명', '특수지구분코드', '특수지구분명', '지번', '기준연도', '기준월',
# '공동주택코드', '공동주택구분코드', '공동주택구분명', '특수지명', '공동주택명', '동명', '층명', '호명',
# '전용면적', '공시가격', '데이터기준일자'],

In [44]:
public = pd.DataFrame()
for path in paths:
    print(path)
    tmp = dd.read_csv(
        path, 
        usecols=['법정동명', '지번', '기준연도', '기준월', '공동주택구분명', '동명', '층명', '호명', '공시가격'],
        dtype={'동명': 'object',
               '지번': 'object',
               '호명': 'object',
               '공시가격': 'float64',
               '기준연도': 'object',
               '기준월': 'float64',
               '층명': 'float64'},
        encoding = 'cp949'
    )
    tmp = tmp[(tmp['기준연도'].isin(['2021']))].compute()
    public = pd.concat([public, tmp], ignore_index=True)
    
public.drop_duplicates(inplace = True)

D:/feature_data/공동주택가격정보\AL_11_D167_20210817.csv
[########################################] | 100% Completed | 45.3s
D:/feature_data/공동주택가격정보\AL_26_D167_20210817.csv
[########################################] | 100% Completed | 19.6s
D:/feature_data/공동주택가격정보\AL_27_D167_20210817.csv
[########################################] | 100% Completed | 12.7s
D:/feature_data/공동주택가격정보\AL_28_D167_20210817.csv
[########################################] | 100% Completed | 17.5s
D:/feature_data/공동주택가격정보\AL_29_D167_20210817.csv
[########################################] | 100% Completed |  9.6s
D:/feature_data/공동주택가격정보\AL_30_D167_20210817.csv
[########################################] | 100% Completed |  9.2s
D:/feature_data/공동주택가격정보\AL_31_D167_20210817.csv
[########################################] | 100% Completed |  6.8s
D:/feature_data/공동주택가격정보\AL_36_D167_20210817.csv
[########################################] | 100% Completed |  1.3s
D:/feature_data/공동주택가격정보\AL_41_D167_20210817.csv
[######        

C:\ProgramData\Anaconda3\envs\xai2\lib\site-packages\dask\core.py:121: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------+---------+----------+
| Column | Found   | Expected |
+--------+---------+----------+
| 공시가격   | float64 | int64    |
| 기준연도   | object  | int64    |
| 기준월    | float64 | int64    |
| 층명     | float64 | int64    |
+--------+---------+----------+

The following columns also raised exceptions on conversion:

- 기준연도
  ValueError("invalid literal for int() with base 10: '2016-11-09'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'공시가격': 'float64',
       '기준연도': 'object',
       '기준월': 'float64',
       '층명': 'float64'}

to the call to `read_csv`/`read_table`.

In [ ]:
public.drop_duplicates(inplace = True)
public.columns = ['dong_address', 'bunji', 'registry_pk', 'base_year', 'base_month','gov_price']    
public = public[public['base_month'] == '01'].reset_index(drop=True)
public['address_dong'] = public['dong_address'] + ' ' + public['bunji'] + '번지'
public['address_dong'] = public['address_dong'].str.strip()
public = public.drop(['dong_address', 'bunji', 'base_month'], axis=1)
print(public.shape)

In [ ]:
df = pd.DataFrame()
for path in paths:
    print(path)
    tmp = dd.read_csv(
        path, 
        usecols=['법정동명', '지번', '기준연도', '기준월', '공동주택구분명', '동명', '층명', '호명', '공시가격'],
        dtype={'동명': 'object',
               '지번': 'object',
               '호명': 'object'},
        encoding = 'cp949'
    )
    tmp = tmp[(tmp['기준연도'].isin(['2021']))].compute()
    df = pd.concat([df, tmp], ignore_index=True)
    

In [ ]:
df.drop_duplicates(inplace = True)
df.columns = ['dong_address', 'bunji', 'registry_pk', 'base_year', 'base_month','gov_price']    
df = df[df['base_month'] == '01'].reset_index(drop=True)
df['address_dong'] = df['dong_address'] + ' ' + df['bunji'] + '번지'
df['address_dong'] = df['address_dong'].str.strip()
df = df.drop(['dong_address', 'bunji', 'base_month'], axis=1)
print(df.shape)

In [18]:
df.columns

Index(['고유번호', '법정동코드', '법정동명', '특수지구분코드', '특수지구분명', '지번', '건축물대장고유번호', '기준연도',
       '기준월', '동코드', '동명', '토지대장면적', '산정대지면적', '건물전체연면적', '건물산정연면적', '주택가격',
       '표준지여부', '데이터기준일자'],
      dtype='object')